In [1]:
import pandas as pd
import numpy as np
import json
import pathlib
pd.options.plotting.backend = "plotly"


In [2]:

# remove invalid filename characters
def slugify(value):
    return "".join(x for x in value if x.isalnum())

def generate_plots(save_path, is_v0):
    # Load data from json
    json_path = save_path / 'profiler_dump.json'
    data = json.load(open(str(json_path)))
    keys = [k for k in data.keys()]

    ## Mean Accuracy Plot
    accumalted_scores = []
    selected_record = data['selected_record']
    for i in range(len(selected_record)):
        accumalted_scores.append(selected_record[i][-2])
    df = pd.DataFrame(accumalted_scores, index=range(len(accumalted_scores)), columns =['y'])
    df.index.name = 'Time'
    y_name = 'Mean accuracy (%)'
    df[y_name] = 100.0*df.rolling(window=MEAN_WINDOW_SIZE).mean()
    fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
    if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
    else: fig.show()
        
    ## Mean Contol elapsed Time Plot
    load_model_ms = data['load_model_ms']
    iter_ms = data['iter_ms']

    contol_elapsed_over_t = []
    model_over_t = []

    for i in range(len(selected_record)):
        inference_ms = selected_record[i][-1]
        contol_elapsed_over_t.append(iter_ms[i] - load_model_ms[i] - inference_ms)
        if is_v0:
            model_over_t.append(lookup[selected_record[i][0]])


    fps_over_t = []
    for i in range(len(iter_ms)):
        fps_over_t.append(1000/iter_ms[i])

    y_name = 'Mean Control Elapsed Time (ms)'
    df = pd.DataFrame(contol_elapsed_over_t, index=range(len(contol_elapsed_over_t)), columns =['y'])
    df.index.name = 'Time'
    df[y_name] = df.rolling(window=MEAN_WINDOW_SIZE).mean()
    fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
    #fig.update_yaxes(range=[0, 3])
    if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
    else: fig.show()

    ## Mean FPS Plot
    y_name = 'Mean FPS'
    df = pd.DataFrame(fps_over_t, index=range(len(fps_over_t)), columns =['y'])
    df.index.name = 'Time'
    df[y_name] = df.rolling(window=MEAN_WINDOW_SIZE).mean()
    fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
    #fig.update_yaxes(range=[0, 80])
    if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
    else: fig.show()

    y_name = 'Load Model elapsed time (ms)'
    df = pd.DataFrame(load_model_ms, index=range(len(load_model_ms)), columns =['y'])
    df.index.name = 'Time'
    #df[y_name] = df.rolling(window=1).mean()
    fig = df.reset_index().plot.line(x='Time', y='y', width=FIG_WIDTH, height=FIG_HEIGHT)
    #fig.update_yaxes(range=[0, 80])
    if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
    else: fig.show()

    if is_v0:
        y_name = 'Selected Model'
        df = pd.DataFrame(model_over_t, index=range(len(model_over_t)), columns =['y'])
        df.index.name = 'Time'
        #df[y_name] = df.rolling(window=1).mean()
        fig = df.reset_index().plot.scatter(x='Time', y='y', width=FIG_WIDTH, height=FIG_HEIGHT)
        #fig.update_yaxes(range=[0, 80])
        if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
        else: fig.show()
    

In [8]:

MEAN_WINDOW_SIZE = 200
FIG_WIDTH= 1000
FIG_HEIGHT = 600

ENABLE_SAVE_IMAGE = True


lookup = {
    'mobilenet_34.67_66.84': 'mobilenet_34.67_66.84',
    'mobilenet_36.04_37.97': 'mobilenet_36.04_37.97',
    'mobilenet_41.75_33.35': 'mobilenet_41.75_33.35',
    'resnet18_39.52_66.54': 'resnet18_39.52_66.54',
    'resnet18_40.87_46.49': 'resnet18_40.87_46.49',
    'resnet18_42.63_15.38': 'resnet18_42.63_15.38',
    'resnet18_58.17_71.87': 'resnet18_58.17_71.87',
    'resnet18_58.66_55.21': 'resnet18_58.66_55.21',
}

paths = [x for x in (pathlib.Path.cwd() / 'jetson-nano-4gb').iterdir() if x.is_dir() and x.name.startswith('jetson_v1')]
#paths = [x for x in (pathlib.Path.cwd() / 'laptop-gtx1650').iterdir() if x.is_dir() and x.name.startswith('laptop_v1')]
for savepath in paths:
    generate_plots(savepath, False)
#savepath.mkdir(parents=True, exist_ok=True)

In [5]:
'''
paths = [x for x in (pathlib.Path.cwd() / 'laptop-gtx1650').iterdir() if x.is_dir() and x.name.startswith('laptop')]
for savepath in paths:
    for ppp in savepath.iterdir():
        if ppp.name.endswith('FPS.png'):
            ppp.unlink()
'''
print('')